In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping

In [2]:
# Dataset load
df1 = pd.read_csv('Dataset/dataset_cleaned1.csv')
df2 = pd.read_csv('Dataset/database_indo1.csv')

In [3]:
df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18787 entries, 0 to 18798
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Maker                                18787 non-null  object 
 1   Model                                18787 non-null  object 
 2   Number_of_Cylinders                  18787 non-null  int64  
 3   Engine_Type                          18787 non-null  int64  
 4   Engine_Horse_Power                   18787 non-null  float64
 5   Engine_Horse_Power_RPM               18787 non-null  int64  
 6   Transmission                         18787 non-null  int64  
 7   Mixed_Fuel_Consumption_per_100_km_l  18787 non-null  float64
 8   Fuel_Tank_Capacity                   18787 non-null  int64  
 9   Acceleration_0_to_100_Km             18787 non-null  float64
 10  Max_Speed_Km_per_Hour                18787 non-null  int64  
 11  Fuel_Grade                       

In [4]:
# Assuming 'Mixed_Fuel_Consumption_per_100_km_l' is the target variable
y = df['Mixed_Fuel_Consumption_per_100_km_l']

# Select specific columns as input features
selected_columns = ['Number_of_Cylinders',
                    'Engine_Type',
                    'Engine_Horse_Power',
                    'Engine_Horse_Power_RPM',
                    'Transmission',
                    'Acceleration_0_to_100_Km',
                    'Fuel_Grade']

X = df[selected_columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
# Build the TensorFlow Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [6]:
# Compile and train the model with the new architecture
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train, epochs=5000, batch_size=1024,
                    validation_split=0.2)

Epoch 1/5000
12/12 [==============================] - 1s 17ms/step - loss: 67.1803 - mae: 7.5257 - val_loss: 62.8936 - val_mae: 7.0550
Epoch 2/5000
12/12 [==============================] - 0s 6ms/step - loss: 49.3240 - mae: 6.2726 - val_loss: 39.3533 - val_mae: 5.2279
Epoch 3/5000
12/12 [==============================] - 0s 6ms/step - loss: 23.5133 - mae: 3.7567 - val_loss: 14.4466 - val_mae: 2.2546
Epoch 4/5000
12/12 [==============================] - 0s 6ms/step - loss: 10.9302 - mae: 2.1222 - val_loss: 13.1463 - val_mae: 2.1552
Epoch 5/5000
12/12 [==============================] - 0s 5ms/step - loss: 9.5333 - mae: 1.8918 - val_loss: 11.6620 - val_mae: 1.8126
Epoch 6/5000
12/12 [==============================] - 0s 6ms/step - loss: 8.8171 - mae: 1.7327 - val_loss: 11.1711 - val_mae: 1.7442
Epoch 7/5000
12/12 [==============================] - 0s 6ms/step - loss: 8.2693 - mae: 1.6759 - val_loss: 10.7254 - val_mae: 1.7144
Epoch 8/5000
12/12 [==============================] - 0s 6ms/ste

In [9]:
# Evaluate on the test set
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f'Test MAE: {test_mae}')

118/118 [==============================] - 0s 2ms/step - loss: 5.5043 - mae: 0.9140
Test MAE: 0.9140150547027588


In [8]:
import joblib

# Save the model to an HDF5 file
model.save('model_dinova1.h5')

# Save the scaler to a joblib file
joblib.dump(scaler, 'scaler_dinova1.joblib')

['scaler_dinova1.joblib']